In [1]:
import pandas as pd

# reading in dataframe and getting credits parameter
df = pd.read_csv('classes_inc.csv')
classes_inc = list(df['Class'])
modified_classes = [c.replace(' ', '_').replace('/', '_').lower().replace('__', '_').replace('comp_sci','cs').replace('e_c_e', 'ece').replace('m_e', 'me').replace('i_sy_e','ie')  for c in classes_inc]
df['ID'] = modified_classes
class_credits = df[['ID', 'Credits']]

In [47]:
ie_requirements_list = ['ie_math221', #take math 221
'math_222', #take math 222
'math_234', #take math 234
'math_340', #take math 340
'stat_311', #choose between stat 311 or 309 
######## (just 311 for now)

'ie_210', #choose between isye210, stat 310, stat 312
######### (just 210 for now)

'cs_220', #take cs 220
'ie_cs_choice', #choose 1 of a few cs classes
'ie_191', #take isye 191
'ie_312', #take isye 312
'ie_313', #take isye 313
'ie_315', #take isye 315
'ie_320', #take isye 320
'ie_321', #take isye 321
'ie_323', #take isye 323
'ie_348', #take isye 348
'ie_psych_349', #take isye/psych 349
'ie_350', #take isye 350
'ie_450', #take isye 450
'ie_phys_ema', #choose 5-6 credits from the following physics/ema
'ie_basic_science', #choose 9 credits from the list of basic science
'ie_an_op', #take 3 of the analytics/operations research
'ie_diff_elective', #take elective not in the analytics/operations area
'ie_elective', #6 credits of any elective
'ie_comm', #3 credits of communication skills
'ie_eng_comm', #must take engineering communication class
'ie_econ', #must take econ 101
]

math_requirements_list = [
# 'math_linalg', #take linear algebra class from options
'math_340', #take linear algebra class from options 
########## (just 340 for now)

'math_algebra', #take 2 analysis/topology/algebra classes
'math_advanced', #take one advanced math class from list,
'math_electives', #take 9 credits of electives
]
    
cs_requirements_list = [
'cs_math_240', #take cs/math 240
'cs_ece_252', #take cs/ece 252
'cs_300', #take cs 300
'cs_ece_354', #take cs/ece 354
'cs_400', #take cs 400
'cs_calc', #take one of the calc sequences
'cs_math', #take 2 courses of additional math 
'cs_theory', #take one theory of CS class
'cs_software', #take two software CS classes,
'cs_app', #take one cs application class
'cs_elective', #take two cs electives 
]

# requirements for all majors 
requirements = cs_requirements_list + ie_requirements_list + math_requirements_list 


d = {'ID': requirements}
temp = pd.DataFrame(data=d) 
required_classes_df = temp.merge(df, on='ID', how='inner')


# classes a student must take, format is (disc, class)
required_classes = [('cs', 'cs_math_240'),
 ('cs','cs_ece_252'),
 ('cs','cs_300'),
 ('cs','cs_ece_354'),
 ('cs','cs_400'),
 ('ie','math_222'),
 ('ie','math_234'),
 ('ie','math_340'),
 ('math','math_340'),
 ('ie','stat_311'),
 ('ie','ie_210'),
 ('ie','cs_220'),
 ('ie','ie_191'),
 ('ie','ie_312'),
 ('ie','ie_313'),
 ('ie','ie_315'),
 ('ie','ie_320'),
 ('ie','ie_321'),
 ('ie','ie_323'),
 ('ie','ie_348'),
 ('ie','ie_psych_349'),
 ('ie','ie_350'),
 ('ie','ie_450')]

requirements

['cs_math_240',
 'cs_ece_252',
 'cs_300',
 'cs_ece_354',
 'cs_400',
 'cs_calc',
 'cs_math',
 'cs_theory',
 'cs_software',
 'cs_app',
 'cs_elective',
 'ie_math221',
 'math_222',
 'math_234',
 'math_340',
 'stat_311',
 'ie_210',
 'cs_220',
 'ie_cs_choice',
 'ie_191',
 'ie_312',
 'ie_313',
 'ie_315',
 'ie_320',
 'ie_321',
 'ie_323',
 'ie_348',
 'ie_psych_349',
 'ie_350',
 'ie_450',
 'ie_phys_ema',
 'ie_basic_science',
 'ie_an_op',
 'ie_diff_elective',
 'ie_elective',
 'ie_comm',
 'ie_eng_comm',
 'ie_econ',
 'math_340',
 'math_algebra',
 'math_advanced',
 'math_electives']

In [48]:
%reload_ext gams.magic
m = gams.exchange_container

In [51]:
# make a set for required classes and electives?
# set for must take classes, list of tuples domain [disc, class]
# set for or take classes domain [disc, req, classes]

classes = m.addSet('classes', records=df['ID'])
semesters = m.addSet('semesters', records=[1, 2, 3, 4, 5, 6, 7, 8])
# reqs = m.addSet('requirements', records=requirements)
disc = m.addSet('discipline', records=['ie', 'cs', 'math'])
musttake = m.addSet('musttake', domain=['discipline', 'classes'], records=required_classes)

credits = m.addParameter('credits', domain='classes', records=class_credits)
max_cred = m.addParameter('max_credits', records=18)
min_cred = m.addParameter('min_credits', records=12)

totclasses = m.addVariable('total_classes', 'free', description = 'total classes')
x = m.addVariable('x', 'binary', [classes, semesters], description='if class c is taken in semester s')
# m.write('tclasses.gdx')

In [54]:
%%gams
# set C "classes" / /;
# set S "semester" /s1*s8/;
# set R "requirements" /ie_math221,ie_math222, ie_math234, ie_math340, ie_phys_ema, ie_basic_science, ie_stat_311_309,
# ie_210_or_stats ,ie_cs220,ie_cs_choice, ie_191, ie_312, ie_313,ie_315,ie_320,ie_321, ie_323, ie_348, ie_349, ie_350, ie_450,
# ie_an_op, ie_diff_elective, ie_elective, ie_comm, ie_eng_comm, ie_econ, math_linalg, math_algebra, math_advanced, math_electives,
# cs_240, cs_252, cs_300, cs_354, cs_400, cs_calc, cs_math, cs_theory, cs_software, cs_app, cs_elective/;
alias(c, classes);
alias(s, semesters);
alias(m, musttake);

# parameter credits(C); #number of credits for each class
# parameter max_cred(S) = 12;
# parameter min_cred(S) = 3;
# parameter req_credits(requirements) /ie_math221 5,ie_math222 4, ie_math234 4, ie_math340 3, ie_phys_ema 5, ie_basic_science 9, ie_stat_311_309 3,
# ie_210_or_stats 3, ie_cs220 4, ie_cs_choice 3, ie_191 2, ie_312 3, ie_313 3,ie_315 3,ie_320 3, ie_321 1, ie_323 3, ie_348 1, ie_349 3, ie_350 3, ie_450 3,
# ie_an_op 9, ie_diff_elective 3, ie_elective 6, ie_comm 6, ie_eng_comm 3, ie_econ 4, math_linalg 3, math_algebra 6, math_advanced 3, math_electives 9,
# cs_240 3, cs_252 3, cs_300 3, cs_354 3, cs_400 3, cs_calc 9, cs_math 6, cs_theory 3, cs_software 6, cs_app 3, cs_elective 6 /;
# table satisfy(C,R)
         
# math_221       1               0.               1
# physics_207    0                1
# chemistry_103  0.                1 ;
   


# set pred(i,j); #classes in i must be taken before class j

# binary variable x(C, S); #1 if class is taken during that semester
# variable total_classes;

equations 
    obj_equation, req_fulfill;

obj_equation..
    totclasses =e= sum((c,s), x(c, s));

# req_fulfill(m)..
#     sum((c, s), x(c, s)) =e= 1;
                         
                         
model triple_major /all/;
solve triple_major using lp minimize total_classes;

#then we do abroad example where only thing that changes is that during s6, no credits are taken

GamsExceptionExecution: GAMS return code not 0 (2), check C:\Users\Rocke\OneDrive\Documents\classes\isye524\project\gj_ec19c19b_base_5.lst for more details

In [ ]:
# requisites = [
#  [('MATH 217', 'MATH 221', 'MATH 275')],
#  [],
#  [('COMP SCI 200', 'COMP SCI 220', 'COMP SCI 302', 'COMP SCI 310', 'COMP SCI 301', 'COMP SCI 300'), 'E C E/COMP SCI  252', 'E C E 203', 'nCOMP SCI 367'],
#  ['E C E/COMP SCI  252',  ('COMP SCI 300', 'COMP SCI 302')],
#  ['COMP SCI 300'],
#  [('COMP SCI 300', 'COMP SCI 367')],
#  ['MATH 222', ('MATH/COMP SCI  240', 'MATH 234'), ('COMP SCI 200','COMP SCI 300','COMP SCI 301', 'COMP SCI 302', 'COMP SCI 310')],
#  [('MATH 320','MATH 340', 'MATH 341', 'MATH 375')],
#  ['STAT/MATH  310', ('STAT 333', 'STAT 340')],
#  [('COMP SCI 367', 'COMP SCI 400'), ('COMP SCI 407','COMP SCI 536','COMP SCI 537','COMP SCI 545','COMP SCI 559','COMP SCI 564','COMP SCI 570','COMP SCI 679','E C E/COMP SCI  552')],
#  [('MATH 340', 'MATH 341','MATH 375'), ('COMP SCI 200', 'COMP SCI 300','COMP SCI 301', 'COMP SCI 302', ' COMP SCI 310')],
#  [('MATH 320', 'MATH 340', 'MATH 341','MATH 375'), ('MATH 322', 'MATH 376', 'MATH 421','MATH 521'), ('COMP SCI 200', 'COMP SCI 220', 'COMP SCI 300', 'COMP SCI 310', 'COMP SCI 300')],
#  [],
#  [('MATH/COMP SCI  240', 'STAT/COMP SCI/MATH  475'), ('COMP SCI 367','COMP SCI 400')],
#  [('COMP SCI 200','COMP SCI 220','COMP SCI 300','COMP SCI 301','COMP SCI 302','COMP SCI 310'), ('MATH 320','MATH 340','MATH 341','MATH 375')],
#  [('MATH 320','MATH 340','MATH 341','MATH 375','MATH 443')],
#  ['STAT/COMP SCI/I SY E/MATH  525', ('COMP SCI 200','COMP SCI 220','COMP SCI 300','COMP SCI 301','COMP SCI 302','COMP SCI 310')],
#  [('MATH 234','MATH 320','MATH 340', 'MATH 341', 'MATH 375'), ('E C E 203', 'COMP SCI 200','COMP SCI 220','COMP SCI 300','COMP SCI 301','COMP SCI 302','COMP SCI 310','COMP SCI 320')],
#  ['E C E 330', ('MATH 320','MATH 340')],
#  [('COMP SCI 300','COMP SCI 367'), ('MATH 217','MATH 221','MATH 275')],
#  ['E C E/COMP SCI  354', ('COMP SCI 367','COMP SCI 400')],
#  ['E C E/COMP SCI  354', ('COMP SCI 367','COMP SCI 400')],
#  [('COMP SCI 200','COMP SCI 220','COMP SCI 300','COMP SCI 301','COMP SCI 302','COMP SCI 310')],
#  [('COMP SCI 300','COMP SCI 320','COMP SCI 367'), ('MATH 211','MATH 217','MATH 221','MATH 275')],
#  [('COMP SCI 400','COMP SCI 320')],
#  [('COMP SCI 320', 'COMP SCI 400')],
#  [('COMP SCI 536','COMP SCI 537','COMP SCI 564')],
#  ['E C E/COMP SCI  352', 'E C E/COMP SCI  354'],
#  [('COMP SCI 367','COMP SCI 400'), 'MATH 234'],
#  [('MATH 222', 'MATH 276'), ('COMP SCI 367','COMP SCI 400')],
#  [('MATH 320','MATH 340','MATH 341','MATH 375', 'M E/COMP SCI/E C E  532','cMATH 320','cMATH 340','cMATH 341','cMATH 375', 'cM E/COMP SCI/E C E  532'), ('E C E 331', 'STAT/MATH  309','STAT/MATH 431', 'STAT 311','STAT 324', 'M E/STAT  424', 'MATH 531')],
#  ['COMP SCI 400', ('MATH 320','MATH 340','MATH 341','MATH 375'), ('STAT 302','STAT 311','STAT 324','STAT 340','STAT 371', 'STAT/MATH  309','STAT/MATH 431', 'MATH 331','MATH 531')],
#  [('MATH 320', 'MATH 340'), ('STAT 511','STAT 541', 'POP HLTH/B M I  551', 'STAT 324','STAT 371', 'STAT/F&W ECOL/HORT  571')],
#  [('COMP SCI 200','COMP SCI 300','COMP SCI 301','COMP SCI 302', 'L I S/COMP SCI  102'), 'nCOMP SCI 270'],
#  ['COMP SCI 400'],
#  [('COMP SCI 320','COMP SCI 400'), 'MATH 222'],
#  [('MATH/COMP SCI  240', 'STAT/COMP SCI/MATH  475'), ('COMP SCI 367','COMP SCI 400')],
#  [('MATH 320','MATH 340','MATH 341','MATH 375'), ('COMP SCI 200','COMP SCI 300','COMP SCI 301','COMP SCI 302', 'L I S/COMP SCI  102')],
#  ['COMP SCI 537'],
#  [('E C E 203', 'E C E 330')],
#  [('MATH 96', 'MATH 171')],
#  ['MATH 171'],

# # MATH 114 or (MATH 112 and 113) or placement into MATH 221. MATH 211 or MATH 213 does not fulfill the requisite
# #############
#  [('MATH 114', ['MATH 112','MATH 113'], 'MATH 221')],
# #############
              
#  [('MATH 217','MATH 221','MATH 275')],
#  [('MATH 222','MATH 276')],
#  [],
#  ['MATH 275'],
#  [('MATH 222','MATH 276')],

# ############
# # MATH 376 and (MATH 234 and 319) or (MATH 234 and 320) or (MATH 234 and 340) or (MATH 234 and 341) or (MATH 234 and 375)
#  ['MATH 376', ['MATH 234','MATH 319'], ['MATH 234','MATH 320'], ['MATH 234','MATH 340'], ['MATH 234','MATH 341'], ['MATH 234','MATH 375']],
# ############
              
#  [('MATH 321','MATH 376')],
#  [('MATH 213', 'MATH 222')],
#  ['MATH 222'],
#  ['MATH 234'],
#  [],
#  ['MATH 375'],
#  [('MATH 234','MATH 376')],

# ############
# # MATH 234 or (MATH 222 and COMP SCI/MATH  240) or MATH 375
#  [('MATH 234', ['MATH 222', 'COMP SCI/MATH  240'], 'MATH 375')],

#  ['MATH 234','MATH 467', ('MATH 322','MATH 341','MATH 376','MATH 421')],
#  [('MATH 234','MATH 375'), ('MATH 320','MATH 340','MATH 341','MATH 375'), ('MATH 341','MATH 375','MATH 421','MATH 467','MATH 521')],
#  ['MATH 541'],
#  [('MATH 234','MATH 376','cMATH 234','cMATH 376'), ('nSTAT/MATH  431', 'nSTAT 311')],
#  [('STAT/MATH  309', 'STAT 311', 'STAT/MATH  431', 'MATH 531'), ('STAT 240', 'STAT 301', 'STAT 302', 'STAT 324', 'STAT 371', 'ECON 310')],
#  [('MATH 234','MATH 376','cMATH 234', 'cMATH 376'), ('nSTAT/MATH  309', 'nSTAT/MATH  431')],
#  [('STAT/MATH  309', 'STAT 311', 'STAT/MATH  431', 'MATH 531')],
#  [('MATH 211','MATH 217','MATH 221','MATH 275'), ('nSTAT 302', 'nSTAT 371')],
#  [('ZOOLOGY/BIOLOGY  101', 'BOTANY/BIOLOGY  130', 'ZOOLOGY/BIOLOGY/BOTANY  151', 'ZOOLOGY 153', 'BIOCORE 381'), ('CHEM 103','CHEM 108','CHEM 109','CHEM 115'), ('nPHYSIOL 435', 'ANAT&PHY 435')],
#  [('nBIOLOGY/ZOOLOGY  101','nBIOLOGY/ZOOLOGY 102', 'nBIOLOGY/BOTANY  130')],
#  ['ZOOLOGY/BIOLOGY/BOTANY  151'],
#  [('MATH 112','MATH 114','MATH 171','MATH 221', 'MATH 211','MATH 221'),('nCHEM 109','nCHEM 115')],
#  ['CHEM 103', ('MATH 112','MATH 114','MATH 171','MATH 221'),('nCHEM 109', 'nCHEM 115')],
#  [('MATH 113','MATH 114','MATH 171', 'MATH 221'), ('nCHEM 104','nCHEM 115')],
#  ['CHEM 115'],
#  [('CHEM 104','CHEM 109','CHEM 116')],
#  [('CHEM 104','CHEM 109'),'nCHEM 329'],
#  [('CHEM 104', 'CHEM 109'), 'nCHEM 327'],
#  [('CHEM 104','CHEM 109','CHEM 116'), ('nCHEM 343','nCHEM 345')],
#  [('CHEM 341','cCHEM 341'), 'nCHEM 344'],
#  [('CHEM 104','CHEM 109','CHEM 116')],
#  [('CHEM 345', 'cCHEM 345'), ('CHEM 102','CHEM 104','CHEM 109','CHEM 116')],
#  ['CHEM 343'],
#  ['CHEM 344','CHEM 345'],
#  [('nCOM ARTS 105','nCOM ARTS 181')],
#  [],
#  ['nCOMP SCI 301'],
#  [('COMP SCI 220', 'COMP SCI 300','COMP SCI 319')],
#  ['nECON 111'],
#  [('MATH 222', 'MATH 276','cMATH 222','cMATH 276')],
#  ['E M A 201'],
#  [('ESL 117', 'ESL 118')],
#  [],
#  [],
#  [('MATH 211','MATH 217','MATH 221','MATH 275')],
#  [('I SY E 210', 'E C E 331', 'MATH/STAT  309', 'STAT 311','STAT 324')],
#  [('MATH 217','MATH 221','MATH 275','cMATH 217','cMATH 221','cMATH 275')],
#  [('I SY E 210', 'E C E 331', 'MATH/STAT  309', 'STAT 311','STAT 324','STAT 371', 'MATH/STAT  431','MATH/STAT  531', 'cI SY E 210', 'cE C E 331', 'cMATH/STAT  309', 'cSTAT 311','cSTAT 324','cSTAT 371', 'cMATH/STAT  431','cMATH/STAT  531')],
#  [('MATH/STAT  309', 'STAT 311', 'MATH/STAT  431','MATH/STAT 531', 'cMATH/STAT  309', 'cSTAT 311', 'cMATH/STAT  431','cMATH/STAT 531'), ('I SY E 210', 'E C E 331', 'STAT/MATH  310', 'STAT 312','STAT 324','STAT 371', 'cI SY E 210', 'cE C E 331', 'cSTAT/MATH  310', 'cSTAT 312','cSTAT 324','cSTAT 371'), ('MATH 320','MATH 340','cMATH 320','cMATH 340')],
#  ['cI SY E 320'],
#  [('MATH 222','MATH 276'), ('MATH 340','MATH 341','MATH 375')],
#  ['cI SY E/PSYCH  349'],
#  [('I SY E 210', 'E C E 331', 'MATH/STAT  309','MATH/STAT 431', 'STAT 311','STAT 324','STAT 371', 'MATH 531', 'PSYCH 210', 'C&E SOC/SOC  360', 'cI SY E 210', 'cE C E 331', 'cMATH/STAT  309','cMATH/STAT 431', 'cSTAT 311','cSTAT 324','cSTAT 371', 'cMATH 531', 'cPSYCH 210', 'cC&E SOC/SOC  360')],
#  [('COMP SCI 200','COMP SCI 220','COMP SCI 300','COMP SCI 301','COMP SCI 302'), ('I SY E 210', 'MATH/STAT  309','MATH/STAT 431', 'MATH 531', 'STAT 311', 'STAT 324'), 'I SY E 315'],
#  [('I SY E 210', 'E C E 331', 'STAT 311','STAT 324', 'MATH/STAT  309','MATH/STAT 431', 'MATH 531')],
#  ['I SY E 315'],
#  ['I SY E 320','I SY E 349'],
#  ['I SY E 313','I SY E 320','I SY E 323','I SY E 350', 'PSYCH/I SY E  349'],
#  [('STAT/MATH  309', 'STAT 311','STAT 224','STAT 324', 'STAT/MATH  431')],
#  ['I SY E 315', ('I SY E 323','E C E/COMP SCI/I SY E  524'), 'I SY E/PSYCH  349'],
#  [('INTEREGR 397', 'cINTEREGR 397')],
#  [('STAT/MATH  309', 'STAT 311', 'STAT/MATH  431')],
#  [('STAT/MATH  309', 'STAT 311', 'STAT/MATH  431'), ('I SY E 323', 'E C E/COMP SCI/I SY E  524')],
#  ['I SY E 315'],
#  [('COMP SCI 200','COMP SCI 220', 'COMP SCI 300'), ('I SY E 323', 'I SY E/COMP SCI/E C E  524'), ('I SY E 210', 'STAT 311','STAT 324', 'STAT/MATH  309','STAT 431')],
#  ['I SY E/PSYCH  349'],
#  ['I SY E/PSYCH  349', 'INTEREGR 397'],
#  ['PSYCH/I SY E  349'],
#  [],
#  [('I SY E 210', 'E C E 331',  'MATH/STAT  310', 'STAT 312','STAT 340')],
#  [('PSYCH/I SY E  349', 'B M E 315')],
#  [('I SY E 210', 'MATH/STAT  310', 'STAT 312', 'cI SY E 210', 'cMATH/STAT  310', 'cSTAT 312')],
#  [],
#  ['I SY E 315'],
#  [],
#  ['I SY E/M E  512'],
#  ['I SY E 315','I SY E 320', 'I SYE 323', ('STAT/MATH  310', 'STAT 312', 'STAT/MATH  431')],
#  ['I SY E 417'],
#  [('COMP SCI 200','COMP SCI 220','COMP SCI 300','COMP SCI 301','COMP SCI 302'), ('STAT 224','STAT 312','STAT 324', 'STAT/MATH  310')],
#  [('STAT/MATH  309,' 'STAT/MATH 311', 'STAT/MATH  431'), ('MATH 320','MATH 340','MATH 341','MATH 375')],
#  [('STAT/MATH  431','STAT/MATH 309', 'STAT 311', 'MATH 531'), ('MATH 320','MATH 340','MATH 341','MATH 375','MATH 421','MATH 531')],
#  [('I SY E 624', 'STAT/I SY E/MATH/OTM  632'), ('COMP SCI 200','COMP SCI 220','COMP SCI 300','COMP SCI 301','COMP SCI 302','COMP SCI 400')],
#  ['I SY E 323', 'I SY E 415'],
#  ['I SY E/PSYCH  349', ('I SY E 210', 'E C E 331', 'MATH/STAT  310', 'STAT 312','STAT 324','STAT 340')],
#  [],
#  ['MATH 234'],
#  ['MATH 521', ('MATH 320','MATH 340','MATH 341','MATH 375')],
#  ['E M A 201', ('MATH 222','MATH 276')],
#  [('CHEM 103','CHEM 108','CHEM 109','CHEM 115'), 'nMICROBIO 303'],
#  [('MICROBIO 101','MICROBIO 303', 'cMICROBIO 101','MICROBIO 303'), 'nMICROBIO 304'],
#  [('MATH 217','MATH 221','MATH 275'), ('cPHYSICS 207','cPHYSICS 247')],
#  [('PHYSICS 103','PHYSICS 201','PHYSICS 207','PHYSICS 247', 'E M A 201'), ('MATH 217','MATH 221','MATH 275'), ('nPHYSICS 208','nPHYSICS 248')],
#  [('PHYSICS 202','PHYSICS 208','PHYSICS 248'), ('nPHYSICS 241','nPHYSICS 244','nPHYSICS 249')],
#  [('MATH 217','MATH 221','MATH 275'), ('nPHYSICS 201','nPHYSICS 247')],
  
# #############
# # PHYSICS 201, 207, 247, E M A 201, or (PHYSICS 103 and MATH 217, 221, or 275). Not open to students with credit for PHYSICS 202 or 248.
#  [('PHYSICS 201','PHYSICS 207','PHYSICS 247', 'E M A 201', ['PHYSICS 103', ('MATH 217','MATH 221','MATH 275')]), ('nPHYSICS 202','nPHYSICS 248')],
# #############
  
#  [('PHYSICS 202','PHYSICS 208','PHYSICS 248'), ('MATH 222','MATH 276'), ('nPHYSICS 205','nPHYSICS 244','nPHYSICS 249')],
#  ['PHYSICS 247', ('MATH 234', 'cMATH 234', 'MATH 376', 'cMATH 376')],
#  ['PHYSICS 248'],
#  ['nBIOLOGY/ZOOLOGY  101','nBIOLOGY/ZOOLOGY 102', 'BIOLOGY/BOTANY  130'],
#  [],
  
# ############
# # MATH 376, (MATH 234 and 319), (MATH 234 and 320), (MATH 234 and 340), (MATH 234 and 341) or (MATH 234 and 375) 
#  ['MATH 376', (['MATH 234', 'MATH 319'], ['MATH 234', 'MATH 320'], ['MATH 234', 'MATH 340'], ['MATH 234','MATH 341'], ['MATH 234','MATH 375'])],
# ############
  
# ############
# # MATH 234, 375, (MATH 222 and COMP SCI/MATH  240), (MATH 222 and 320), or (MATH 222 and 340)
#  ['MATH 234','MATH 375', (['MATH 222', 'COMP SCI/MATH  240'], ['MATH 222','MATH 320'], ['MATH 222','MATH 340'])],
# ############
  
#  [],
#  [('MATH 320','MATH 340','MATH 341','MATH 375'), ('MATH 322','MATH 376','MATH 421','MATH 521')],
#  [('MATH 376','MATH 421','MATH 521')],
#  [('MATH 320','MATH 340','MATH 341','MATH 375', 'M E/COMP SCI/E C E  532'), ('STAT/MATH  309','STAT/MATH 431', 'MATH 531', 'STAT 311', 'E C E 331'), ('MATH 322','MATH 341','MATH 375','MATH 421','MATH 467', 'COMP SCI 577')],
#  ['MATH 551','MATH 541'],
  
# ############
# # (MATH 234 and 467) or (MATH 341, 375, 421 or 521)
#  [(['MATH 234','MATH 467'], ('MATH 341','MATH 375','MATH 421','MATH 521'))],
# ############
  
#  ['MATH 415', ('MATH 320','MATH 340','MATH 341','MATH 375')],
#  [('MATH 322','MATH 421','MATH 521'), ('MATH 319','MATH 320','MATH 376','MATH 415','MATH 519')],
#  ['MATH 522', 'MATH 521', 'MATH 561'],           
#  ['MATH 321'],
#  ['MATH 521'],
#  ['MATH 522'],
#  ['MATH 521', 'STAT/I SY E/MATH/OTM  632']]

# class_reqs = {}
# for i, c in enumerate(requisites):
#     cl = list(df['Class'])[i]
#     class_reqs[cl] = c
# class_reqs